# Assignment 5 – Prescriptive Analytics: Linear Optimisation

In [43]:
from datetime import datetime, timedelta
print(f'Updated {datetime.now().strftime("%d.%m.%Y")} / Tatu Erkinjuntti')

Updated 25.11.2025 / Tatu Erkinjuntti


## Learning Objectives

This assignment introduces linear optimisation and how to use optimisation in decision-making. In doing so, you will also learn how to present information clearly and how to document and explain calculation steps created in notebooks.



## Initial information

The company manufactures two products, a grate and a stirrer. The grate has a margin of €250 and the stirrer a margin of €320 per product. There are four stages in the manufacturing process: cutting, punching, pressing, and painting.

Each work stage requires the following amount of time (minutes) from the products:

<table align="center">
<tr>
<th></th>
<th>cutting</th>
<th>punching</th>
<th>pressing</th>
<th>painting</th>
</tr>

<tr>
<th>grate</th>
<td>12</td>
<td>16</td>
<td>12</td>
<td>22</td>
</tr>

<tr>
<th>stirrer</th>
<td>20</td>
<td>7</td>
<td>6</td>
<td>15</td>
</tr>

<tr>
<th>capasity</th>
<td>25&nbsp;000</td>
<td>30&nbsp;000</td>
<td>30&nbsp;000</td>
<td>20&nbsp;000</td>
</tr>

</table>

## Matters to be found out

Find out:

1. Optimal production program. (20 %)
2. Maximum margin. (20 %)
3. The company has €25,000 at its disposal, which it can use to increase capacity at any stage of production, but only at one stage, at a cost of €1/min. How should the company use the €25,000 to increase capacity? (30 %)
4. In relation to the initial situation (the additional capacity of €25,000 is not used in this case), the company still has a product called jiggery-pokery, which has a margin of €350 per product and requires the following capacity:
<table>
<tr>
<th></th>
<th>cutting</th>
<th>punching</th>
<th>pressing</th>
<th>painting</th>
</tr>
<tr>
<th>jiggery-pokery</th>
<td>30</td>
<td>20</td>
<td>10</td>
<td>25</td>
</tr>
</table>
What is the company's optimal production and corrsponding maximum profit margin?  (30 %)

In [44]:
# Before we can begin, import necessary libraries.

from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

### Assignment part 1 and 2

We start off by finding out the optimal production program for the grates and stirrers and based on these, determine the maximum margin.

In [45]:
# Create the model
model = LpProblem(name="optimal-production", sense=LpMaximize)

# Define variables (the number of grates and stirrers to produce)
grate_production = LpVariable(name="Grate", lowBound=0)
stirrer_production = LpVariable(name="Stirrer", lowBound=0)

# Define the constraints based on each production stage
model += (12 * grate_production + 20 * stirrer_production <= 25000, "cutting_capacity")
model += (16 * grate_production + 7 * stirrer_production <= 30000, "punching_capacity")
model += (12 * grate_production + 6 * stirrer_production <= 30000, "pressing_capacity")
model += (22 * grate_production + 15 * stirrer_production <= 20000, "painting_capacity")

# Define the objective function (Maximize margin)
model += lpSum([250 * grate_production, 320 * stirrer_production])


Lets check the model.

In [46]:
model

optimal-production:
MAXIMIZE
250*Grate + 320*Stirrer + 0.0
SUBJECT TO
cutting_capacity: 12 Grate + 20 Stirrer <= 25000

punching_capacity: 16 Grate + 7 Stirrer <= 30000

pressing_capacity: 12 Grate + 6 Stirrer <= 30000

painting_capacity: 22 Grate + 15 Stirrer <= 20000

VARIABLES
Grate Continuous
Stirrer Continuous

**NOTE:** since the assignment does not specify to use integer optimisation (Integer Linear Programming), we will stick to the default continuous optimisation.

Lets optimise.

In [47]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/3j/zrw9gdxj61g3x4ypt77qk04h0000gp/T/b42579bd3d644ae287bcbf2da2462ad2-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/3j/zrw9gdxj61g3x4ypt77qk04h0000gp/T/b42579bd3d644ae287bcbf2da2462ad2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 20 RHS
At line 25 BOUNDS
At line 26 ENDATA
Problem MODEL has 4 rows, 2 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-1) rows, 2 (0) columns and 6 (-2) elements
0  Obj -0 Dual inf 570 (2)
2  Obj 405576.92
Optimal - objective value 405576.92
After Postsolve, objective 405576.92, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 405576.9231 - 2 iterations time 0.002, Pres

1

Lets check the results.

In [48]:
# Status

print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [49]:
for var in model.variables():
    print(f"Optimal number for {var.name} production is {var.value():.0f}")

Optimal number for Grate production is 96
Optimal number for Stirrer production is 1192


In [50]:
print(f"Maximum Margin is {model.objective.value():,.2f} €")

Maximum Margin is 405,576.93 €


#### Assignment part 1 and 2 summary

By using continuous optimisation we can determine that the optimal production numbers for grates is 96 and for the stirrers 1192.

Maximum margin is 405,576.93 €

### Assignment part 3

Since the the company has €25,000 at its disposal, how should they use it to increase capacity?


In [51]:
# Define variables

investment = 25000
cost_per_min = 1
increase_minutes = investment / cost_per_min
# note that in this specific case this calculation is unnecessary, since the investment equates to increased minutes in production, but if the cost per minute increases, this comes in handy.

print(f"The company can increase capacity by {increase_minutes:.0f} minutes in one stage.")

The company can increase capacity by 25000 minutes in one stage.


In [52]:
# Since we are comparing different stages, I determine that looping through the stages is the leanest approach.

# Define a dictionary for results and results comparison.
investment_results = {}

# Iterate through each stage to evaluate the investment
for production_stage in ["cutting", "punching", "pressing", "painting"]:
    # Define a model
    investment_model = LpProblem(name="Production_investment", sense=LpMaximize)

    # Define variables
    grate_production_investment = LpVariable(name="Grate", lowBound=0)
    stirrer_production_investment = LpVariable(name="Stirrer", lowBound=0)

    # Define objective function
    investment_model += lpSum([250 * grate_production_investment, 320 * stirrer_production_investment])

    # Define constraints with the increased capacity for the selected stage
    investment_model += (12 * grate_production_investment + 20 * stirrer_production_investment <= 25000 + (increase_minutes if production_stage == 'cutting' else 0), "cutting_capacity")
    investment_model += (16 * grate_production_investment + 7 * stirrer_production_investment <= 30000 + (increase_minutes if production_stage == 'punching' else 0), "punching_capacity")
    investment_model += (12 * grate_production_investment + 6 * stirrer_production_investment <= 30000 + (increase_minutes if production_stage == 'pressing' else 0), "pressing_capacity")
    investment_model += (22 * grate_production_investment + 15 * stirrer_production_investment <= 20000 + (increase_minutes if production_stage == 'painting' else 0), "painting_capacity")

    # Optimise this iteration.
    investment_model.solve()

    # Store the results to the dictionary
    investment_results[production_stage] = investment_model.objective.value()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/3j/zrw9gdxj61g3x4ypt77qk04h0000gp/T/37e85102664847149c87f3018b19d1b2-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/3j/zrw9gdxj61g3x4ypt77qk04h0000gp/T/37e85102664847149c87f3018b19d1b2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 20 RHS
At line 25 BOUNDS
At line 26 ENDATA
Problem MODEL has 4 rows, 2 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-1) rows, 2 (0) columns and 6 (-2) elements
0  Obj -0 Dual inf 570 (2)
1  Obj 426666.67
Optimal - objective value 426666.67
After Postsolve, objective 426666.67, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 426666.6667 - 1 iterations time 0.002, Pres

In [53]:
# Print the results of each investment scenarios.
print("Margin results for investment scenarios")
for stage, margin in investment_results.items():
    print(f"Investing {investment:,.2f} € in stage {stage} produces a {margin:,.2f} € margin.")

# Find the most productive stage to invest in
productive_stage = max(investment_results, key=investment_results.get)
productive_margin = investment_results[productive_stage]

print(f"\nThe most productive stage to invest in is {productive_stage}")
print(f"Resulting maximum margin: €{productive_margin:,.2f}")

Margin results for investment scenarios
Investing 25,000.00 € in stage cutting produces a 426,666.66 € margin.
Investing 25,000.00 € in stage punching produces a 405,576.93 € margin.
Investing 25,000.00 € in stage pressing produces a 405,576.93 € margin.
Investing 25,000.00 € in stage painting produces a 504,449.16 € margin.

The most productive stage to invest in is painting
Resulting maximum margin: €504,449.16


#### Assignment part 3 summary

By using continuous optimisation we can determine that the best choice for the investment would be *painting*.

**Note:** I am not adding the monetary values in this, since they are variables that might change. Monetary results can be viewed in the previous print out.

### Assignment part 4

Lets investigate how the third product jiggery-pokery fits in with the others. As specified in the assignment, investment in part 3 is not realised in this assignment.


In [54]:
# In this assignment we can utilize much of the previous code done in part 1 and 2.

# Create the model
new_model = LpProblem(name="new-optimal-production", sense=LpMaximize)

# Define variables (the number of grates and stirrers to produce)
new_grate_production = LpVariable(name="new_grate", lowBound=0)
new_stirrer_production = LpVariable(name="new_stirrer", lowBound=0)
jiggery_production = LpVariable(name="jiggery-pokery", lowBound=0)

# Define the objective function (Maximize margin)
new_model += lpSum([250 * new_grate_production, 320 * new_stirrer_production, 350 * jiggery_production])

# Define the constraints based on each production stage
new_model += (12 * new_grate_production + 20 * new_stirrer_production + 30 * jiggery_production <= 25000, "cutting_capacity")
new_model += (16 * new_grate_production + 7 * new_stirrer_production + 20 * jiggery_production <= 30000, "punching_capacity")
new_model += (12 * new_grate_production + 6 * new_stirrer_production + 10 * jiggery_production <= 30000, "pressing_capacity")
new_model += (22 * new_grate_production + 15 * new_stirrer_production + 25 * jiggery_production <= 20000, "painting_capacity")

# check model
new_model

new-optimal-production:
MAXIMIZE
350*jiggery_pokery + 250*new_grate + 320*new_stirrer + 0.0
SUBJECT TO
cutting_capacity: 30 jiggery_pokery + 12 new_grate + 20 new_stirrer <= 25000

punching_capacity: 20 jiggery_pokery + 16 new_grate + 7 new_stirrer <= 30000

pressing_capacity: 10 jiggery_pokery + 12 new_grate + 6 new_stirrer <= 30000

painting_capacity: 25 jiggery_pokery + 22 new_grate + 15 new_stirrer <= 20000

VARIABLES
jiggery_pokery Continuous
new_grate Continuous
new_stirrer Continuous

In [55]:
new_model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/3j/zrw9gdxj61g3x4ypt77qk04h0000gp/T/8bcf4403e4dc470388d2756d782df02c-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/3j/zrw9gdxj61g3x4ypt77qk04h0000gp/T/8bcf4403e4dc470388d2756d782df02c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 25 RHS
At line 30 BOUNDS
At line 31 ENDATA
Problem MODEL has 4 rows, 3 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-1) rows, 3 (0) columns and 9 (-3) elements
0  Obj -0 Dual inf 1114.0909 (3)
3  Obj 405576.92
Optimal - objective value 405576.92
After Postsolve, objective 405576.92, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 405576.9231 - 3 iterations time 0.00

1

In [56]:
print(f"status: {new_model.status}, {LpStatus[new_model.status]}")

for var in new_model.variables():
    print(f"Optimal number for {var.name} production is {var.value():.0f}")

print(f"Maximum Margin is {new_model.objective.value():,.2f} €")

status: 1, Optimal
Optimal number for jiggery_pokery production is 0
Optimal number for new_grate production is 96
Optimal number for new_stirrer production is 1192
Maximum Margin is 405,576.93 €


#### Assignment part 4 summary

By using continuous optimisation we can determine the *jiggery-pokery* product is too expensive to produce, even if it has a higher sales price. To achieve the maximum margin of 405,576.93 €, none (0) should be produced.

This leaves the optimal production numbers for grates is 96 and for the stirrers 1192.